<a href="https://colab.research.google.com/github/Xeesto/UEP/blob/dev/Kolokwium_2023_termin0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
# Ustaw wersję jako parametr
SPARK_VERSION="3.5.6"

# Instalacja OpenJDK 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Pobranie Apache Spark z określoną wersją
!wget -q http://www.apache.org/dist/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop3.tgz

# Rozpakowanie archiwum Spark
!tar xf spark-$SPARK_VERSION-bin-hadoop3.tgz

# Instalacja findspark i pyspark
!pip install -q findspark==1.3.0
!pip install -q pyspark==$SPARK_VERSION

# Ustalamy zmienne środowiskowe.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{SPARK_VERSION}-bin-hadoop3"

In [96]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

findspark.init(f"spark-{SPARK_VERSION}-bin-hadoop3")
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName('abc').getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-7ffd166e450c>:7 

**ZADANIE 1** ------------------------------------------

In [97]:
df = spark.read.csv('all_weekly_excess_deaths.csv', header=True, inferSchema=True, sep=';')

In [98]:
df.show()

+---------+---------+-----------+----------+----------+----+----+----+----------+------------+------------+----------------+----------------+----------------+---------------------+----------------------+------------------------+
|  country|   region|region_code|start_date|  end_date|days|year|week|population|total_deaths|covid_deaths| expected_deaths|   excess_deaths|non_covid_deaths|covid_deaths_per_100k|excess_deaths_per_100k|excess_deaths_pct_change|
+---------+---------+-----------+----------+----------+----+----+----+----------+------------+------------+----------------+----------------+----------------+---------------------+----------------------+------------------------+
|Australia|Australia|          0|01/01/2020|07/01/2020|   7|2020|   1|  25734100| 2497 deaths|         0.0|2463.11165730355|33.8883426964494|          2497.0|                  0.0|     0.131686527589655|      0.0137583461131225|
|Australia|Australia|          0|08/01/2020|14/01/2020|   7|2020|   2|  25734100| 25

In [99]:
from pyspark.sql.functions import col, substring,split

In [100]:
# Podziel tekst w kolumnie total_deaths na dwie części: przed i po pierwszej spacji
split_col = split(col("total_deaths"), " ", 2)

# Nowa kolumna z częścią przed spacją
df = df.withColumn("total_deaths_number", split_col.getItem(0))

In [130]:
df.show()

+---------+---------+-----------+----------+----------+----+----+----+----------+------------+------------+----------------+----------------+----------------+---------------------+----------------------+------------------------+-------------------+
|  country|   region|region_code|start_date|  end_date|days|year|week|population|total_deaths|covid_deaths| expected_deaths|   excess_deaths|non_covid_deaths|covid_deaths_per_100k|excess_deaths_per_100k|excess_deaths_pct_change|total_deaths_number|
+---------+---------+-----------+----------+----------+----+----+----+----------+------------+------------+----------------+----------------+----------------+---------------------+----------------------+------------------------+-------------------+
|Australia|Australia|          0|01/01/2020|07/01/2020|   7|2020|   1|  25734100| 2497 deaths|         0.0|2463.11165730355|33.8883426964494|          2497.0|                  0.0|     0.131686527589655|      0.0137583461131225|               2497|
|Aus

In [102]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- days: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- total_deaths: string (nullable = true)
 |-- covid_deaths: double (nullable = true)
 |-- expected_deaths: double (nullable = true)
 |-- excess_deaths: double (nullable = true)
 |-- non_covid_deaths: double (nullable = true)
 |-- covid_deaths_per_100k: double (nullable = true)
 |-- excess_deaths_per_100k: double (nullable = true)
 |-- excess_deaths_pct_change: double (nullable = true)
 |-- total_deaths_number: string (nullable = true)



In [103]:
df = df.withColumn("total_deaths_number", col("total_deaths_number").cast("int"))

In [104]:
# Napisałem samemu - potrzebne grupowanie po regionie
df_total = df.select('total_deaths_number', 'country', 'region').orderBy('total_deaths_number', ascending=False)

In [105]:
df_total.show()

+-------------------+-------------+-------------+
|total_deaths_number|      country|       region|
+-------------------+-------------+-------------+
|              86388|United States|United States|
|              85319|United States|United States|
|              84896|United States|United States|
|              84420|United States|United States|
|              83054|United States|United States|
|              82186|United States|United States|
|              81746|United States|United States|
|              79114|United States|United States|
|              77568|United States|United States|
|              77003|United States|United States|
|              76815|United States|United States|
|              74346|United States|United States|
|              73906|United States|United States|
|              73536|United States|United States|
|              72288|United States|United States|
|              72025|United States|United States|
|              69306|United States|United States|


In [148]:
from pyspark.sql.functions import max

df_total = (
       df.select('total_deaths_number', 'country', 'region')
      .groupBy('region')
      .agg(sum('total_deaths_number').alias('all_week_deaths'))
      .orderBy('all_week_deaths', ascending=False)
      .limit(5)
)

df_total.show()

+-------------+---------------+
|       region|all_week_deaths|
+-------------+---------------+
|United States|        4443319|
|       Mexico|        1389879|
|      Germany|        1327556|
|      Britain|         946967|
|       France|         873266|
+-------------+---------------+



In [149]:
from pyspark.sql.functions import lit
df_top_deaths = df_total.withColumn("tuple_added", lit("TOTAL DEATHS IN 2020"))

In [150]:
from pyspark.sql import functions as F

In [151]:
# Dodanie kolumny ze średnią zgonów
df_mean = (
    df.groupBy('region')
      .agg(F.round(F.mean('total_deaths_number'), 2).alias('avg_last_week_deaths'))
)

In [152]:
df_mean.show()

+--------------------+--------------------+
|              region|avg_last_week_deaths|
+--------------------+--------------------+
|                Utah|               427.9|
|              Hawaii|              232.93|
|    Pays de la Loire|              715.57|
|           Minnesota|              981.68|
|              Madrid|             1043.63|
|              Sweden|             1836.54|
|                Ohio|             2777.99|
|            Coquimbo|               93.97|
|            Asturias|              275.75|
|           Balearics|              163.72|
|             Britain|             13528.1|
|              Biobio|              290.91|
|             Corsica|               63.96|
|              Aragón|              257.49|
|            Lombardy|             2530.57|
|             Germany|            19239.94|
|Bourgogne-Franche...|               652.3|
|            Arkansas|              737.31|
|              Oregon|              780.54|
|           Cantabria|          

In [153]:
# Złącznie z poprzednim wynikiem
df_total_with_avg = df_total.join(df_mean, on='region', how='left')

In [154]:
df_total_with_avg.show()

+-------------+---------------+--------------------+
|       region|all_week_deaths|avg_last_week_deaths|
+-------------+---------------+--------------------+
|United States|        4443319|            65342.93|
|       Mexico|        1389879|            21382.75|
|      Germany|        1327556|            19239.94|
|      Britain|         946967|             13528.1|
|       France|         873266|            12656.03|
+-------------+---------------+--------------------+



In [155]:
df_min_max = df.groupBy('region').agg(
    F.min('total_deaths_number').alias('min_total_deaths'),
    F.max('total_deaths_number').alias('max_total_deaths')
)

df_min_max.show()

+--------------------+----------------+----------------+
|              region|min_total_deaths|max_total_deaths|
+--------------------+----------------+----------------+
|                Utah|             364|             557|
|              Hawaii|             200|             281|
|    Pays de la Loire|             236|             874|
|           Minnesota|             797|            1406|
|              Madrid|             257|            4599|
|              Sweden|            1486|            2569|
|                Ohio|            2363|            4245|
|            Coquimbo|              19|             131|
|            Asturias|              79|             437|
|           Balearics|              50|             224|
|             Britain|            9023|           24691|
|              Biobio|              96|             377|
|             Corsica|              16|              89|
|              Aragón|              86|             458|
|            Lombardy|         

In [156]:
df_total_with_min_max = df_total_with_avg.join(df_min_max, on='region', how='left')

In [157]:
df_total_with_min_max.show()

+-------------+---------------+--------------------+----------------+----------------+
|       region|all_week_deaths|avg_last_week_deaths|min_total_deaths|max_total_deaths|
+-------------+---------------+--------------------+----------------+----------------+
|United States|        4443319|            65342.93|           50995|           86388|
|       Mexico|        1389879|            21382.75|           13170|           43652|
|      Germany|        1327556|            19239.94|           16166|           25493|
|      Britain|         946967|             13528.1|            9023|           24691|
|       France|         873266|            12656.03|            3993|           18767|
+-------------+---------------+--------------------+----------------+----------------+

